In [1]:
import json
import pandas as pd
import os
from dateparser.search import search_dates


In [2]:
os.chdir("../")
os.getcwd()

'/Users/shorouqza/Code/Wikimpacts'

In [3]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.max_columns', None)

In [4]:
def get_date(x):
    if isinstance(x, str):
        return x
    if isinstance(x, list):
        return x[0]
    elif isinstance(x, dict):
        normalized_x = {}
        for k,v in x.items():
            normalized_x[k.strip().lower()] = str(v) 
        day, month, year, date, time = None, None, None, None, None
        if "year" in normalized_x.keys(): 
            if "year" in normalized_x.keys():
                year = normalized_x["year"]
            if "month" in normalized_x.keys():
                month = normalized_x["month"]
               
            if "day" in normalized_x.keys():
                day = normalized_x["day"]
               
        elif "date" in normalized_x.keys():
            date = normalized_x["date"]
        elif "time" in normalized_x.keys():
            time = normalized_x["time"]
        else:
            for k in normalized_x.keys():
                text = search_dates(normalized_x[k], settings={'STRICT_PARSING': False, 'DATE_ORDER': 'DMY'})
                for x in text:
                    if x:
                        date_string =  x[0]
                        date = date_string
                        break   
    
    if year and month and day:
        result = f"{day}-{month}-{year}"
    elif year and month:
        result = f"{month}-{year}"
    elif year:
        result = year
    elif time: result = time
    elif date: result = date # prefer date if present
    else: 
        result = normalized_x
        print(normalized_x)
    return result

In [5]:
import pprint

#model = "mixtral-8x7B-dev"
model = "mistral-7b"

#file_list_path = f"Database/raw/nlp4climate/{model}-json"
# file_list_path = f"Database/raw/nlp4climate/mistral_outputs/dev/Mistral-7B-Instruct-v0.2"
# file_list_path = "Database/raw/nlp4climate/mistral_outputs/test/Mixtral-8x7B-Instruct-v0.1"
file_list_path = "Database/raw/nlp4climate/mistral_outputs/test/Mistral-7B-Instruct-v0.2"

file_list = os.listdir(file_list_path)
file_list_relative = [f"{file_list_path}/{i}" for i in file_list if i and i.endswith(".json")]

dfs = []
for idx in range(len(file_list_relative)):
    try:
        json_file = json.load(open(file_list_relative[idx]))
        country_col = None
        if json_file:
            columns = list(json_file.keys())
            country_col = [i for i in columns if i.lower().strip().startswith("countr") and not i.lower().strip().endswith("annotation")]
            #print(country_col)
            if len(country_col) >= 1:
                country_col = country_col[0]
            else:
                country_col = None
        
        if country_col:
            # print(json_file[country_col])
            try:
                if len(json_file[country_col])>=1 and isinstance(json_file[country_col][0], str):
                    json_file["Country_Merge"] = json_file[country_col]
                    #print("ready country col")
                elif len(json_file[country_col])>=1 and isinstance(json_file[country_col], list):
                    if isinstance(json_file[country_col][0], dict) and ("Country" in json_file[country_col][0] or "country" in json_file[country_col][0]):
                        #print("DICTIONARY")
                        json_file["Country_Merge"] = [d["Country"] for d in json_file[country_col]]
                        #print("EXTRACTED", json_file["Country_Merge"])
                    elif isinstance(json_file[country_col][0], dict) and ("Country" not in json_file[country_col][0] and "country" not in json_file[country_col][0]):
                        json_file["Country_Merge"] = list(json_file[country_col].keys())
            except:
                json_file["Country_Merge"] = []

        json_file["Event_Name"] = file_list[idx].split(".json")[0]
        if "Start_Date" in json_file.keys():
            start_date = get_date(json_file["Start_Date"])
            json_file["Start_Date"] = start_date
        if "End_Date" in json_file.keys():
            end_date = get_date(json_file["End_Date"])
            json_file["End_Date"] = end_date

        dfs.append(json_file)
        print()
    
    except BaseException as err:
        print("----")
        pprint.pprint(json_file)
        print(type(json_file))
        print(err)
        # print(json_file)
        # print(json_file["Start_Date"])
        # print("country_col", country_col, json_file[country_col])
        print()

# df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.




In [6]:
len(dfs)

180

In [7]:
mixtral = pd.DataFrame(dfs, columns=
                             [
                                 "Event_ID",
                                "Event_Name",
                                 "Main_Event",
                                 "Main_Event_Assessment_With_Annotation",
                                 "Start_Date",
                                 "End_Date",
                                 "Time_with_Annotation",
                                 "Country_Merge",
                                 "Countries_Affected",
                                 "Country_with_Annotation",
                                 "Total_Summary_Death",
                                 "Specific_Instance_Per_Country_Death",
                             ])

In [8]:
# event_ids = pd.read_parquet("Database/output/nlp4climate/dev_set.parquet", engine="pyarrow")[["Event_Name", "Event_ID"]]
# event_ids = pd.read_json("Database/raw/nlp4climate/mistral-7b.json")[["Event_Name", "Event_ID"]]

In [9]:
event_ids = pd.read_json("Database/raw/nlp4climate/event_ids.json")

In [10]:
# event_ids["Event_Name"] = event_ids["Event_Name"].apply(lambda x: x.strip().replace(" ", "_"))

In [11]:
#mixtral_ids = pd.merge(
#    left=mixtral,
#    right=event_ids,
#    how="left"
#)

In [12]:
# mixtral_ids.sort_index(inplace=True)

In [13]:
#mixtral_ids.shape
mixtral.shape

(180, 12)

In [14]:
# mixtral_ids[mixtral_ids["Event_ID"].isna()]

mixtral[mixtral["Event_ID"].isna()]

,Event_ID,Event_Name,Main_Event,Main_Event_Assessment_With_Annotation,Start_Date,End_Date,Time_with_Annotation,Country_Merge,Countries_Affected,Country_with_Annotation,Total_Summary_Death,Specific_Instance_Per_Country_Death


In [15]:
# mixtral_ids.to_csv(f"Database/raw/nlp4climate/mistral_outputs/dev/{model}.csv")
# mixtral_ids.to_json(f"Database/raw/nlp4climate/mistral_outputs/dev/{model}.json", orient="records")
mixtral.to_json(f"Database/raw/nlp4climate/mistral_outputs/test/{model}.json", orient="records")

In [ ]:
mixtral.head(3)

####